# Validación metodología CORFO

El objetivo de este notebook es validar cuantitativa y cualitativamente la metodología empleada para clasificar los proyectos CORFO, donde se empleó el modelo de lenguaje GPT 4 para clasificar cada proyecto. Para ello, se hará una etiquetarán manualmente algunos proyectos, y se analizará la concordancia entre las etiquetas manuales y aquellas provistas por GPT.

In [1]:
import os
import math
import requests
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import itertools
from google.colab import drive
from sklearn.metrics import classification_report

drive.mount('/content/drive')

Mounted at /content/drive


## Abrir datos

In [2]:
data_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/CORFO/Datos"
results_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/CORFO/Resultados"
df_corfo = pd.read_excel(os.path.join(data_dir, "DataInnovacion - InnovaChile Corfo.xlsx"),  skiprows=1)
df_corfo

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Inicio Actividad Económica,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final
0,23SN-242063,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77003755-7,Bebidas Saludables SpA,Mejorar las capacidades de las bebidas Masala ...,...,2019-05-20 00:00:00,Turismo,Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,Sí,NaN,NaN,Sin tendencia
1,23SN-242058,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,76228136-8,Tecagromed SpA,TECFILMS: Desarrollo de un biomaterial en form...,...,2012-11-20 00:00:00,Comercio y retail,Intercambiar,Economía Circular,R2 Reducir,Uso y fabricación de productos más inteligentes,No,NaN,NaN,Química Verde (Green Chemestry)
2,23SN-242042,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77618332-6,Green Focus Solutions SpA,Kit molecular para la identificación rápida de...,...,2022-12-08 00:00:00,Biotecnológico,NaN,Producción Sostenible,NaN,NaN,No,NaN,NaN,Nanotecnología
3,23SN-242041,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77704466-4,Zeroglut SpA,Evaluación de tecnologías de envasado para inc...,...,2023-02-21 00:00:00,Alimentos (excepto vitivinícola),Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,No,NaN,NaN,Sin tendencia
4,23SN-242036,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77147033-5,La mar spa,Desarrollo de un pastel de jaibas listo para c...,...,2020-04-23 00:00:00,Comercio y retail,Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,No,NaN,NaN,Sin tendencia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9439,09ANT-7178,Consolida y Expande,Subsidio,Apoyo Negocios Tecnológicos,Apoyo Negocios Tecnológicos,FINALIZADO,Persona Jurídica constituida en Chile,99576630-2,OXZO SOCIEDAD ANONIMA,introducción al mercado nacional de plataforma...,...,2004-12-01 00:00:00,Manufactura de no metálicos,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9440,09ANT-7131,Consolida y Expande,Subsidio,Apoyo Negocios Tecnológicos,Apoyo Negocios Tecnológicos,FINALIZADO,Persona Jurídica constituida en Chile,77183150-8,METALCOP INGENIERIA LIMITADA,nuevo sistema apertura tapa balde para palas d...,...,1998-07-10 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9441,09ACS-7457,Desarrolla innovación,Subsidio,Apoyo Elaboración Capital Semilla,Capital Semilla,FINALIZADO,Persona Jurídica constituida en Chile,96691330-4,DICTUC S.A.,apoyo a la elaboración de estudios de preinver...,...,1994-03-02 00:00:00,Servicios de ingeniería o de conocimiento,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9442,09AAP-6741,Renuévate,Subsidio,Apoyo A La Atracción De Profesionales Y Técnicos,Capital Humano,FINALIZADO,Persona Jurídica constituida en Chile,76832220-1,ORION FORESTAL SA,desarrollo de sistemas de recolección de bioma...,...,2007-05-11 00:00:00,Forestal,Intercambiar,Economía Circular,NaN,Uso y fabricación de productos más inteligentes,No,NaN,Biocombustible,Sin tendencia


In [3]:
df_class = pd.read_csv(os.path.join(results_dir, "clasificaciones_corfo_enfermedades_especificas_limpia.csv"))
df_class

,código,glosa,clasificación,clasificación_normalizada
0,23SN-242063,TÍTULO: Mejorar las capacidades de las bebidas...,otro,otro
1,23SN-242058,TÍTULO: TECFILMS: Desarrollo de un biomaterial...,otro,otro
2,23SN-242042,TÍTULO: Kit molecular para la identificación r...,otro,otro
3,23SN-242041,TÍTULO: Evaluación de tecnologías de envasado ...,otro,otro
4,23SN-242036,TÍTULO: Desarrollo de un pastel de jaibas list...,otro,otro
...,...,...,...,...
9437,10SUP-8805,TÍTULO: entrustet. OBJETIVO: entrustet is a fr...,otro,otro
9438,10SUP-8800,TÍTULO: assertid - online identity verificatio...,otro,otro
9439,10SUP-8780,TÍTULO: h2020. OBJETIVO: it is estimated that ...,otro,otro
9440,10SUP-8778,TÍTULO: androidim (code name). OBJETIVO: who a...,otro,otro


In [4]:
df_health = pd.read_csv(os.path.join(results_dir, "clasificaciones_corfo_salud_otro_limpia.csv"))
df_health


,código,glosa,clasificación,clasificación_normalizada
0,18IS6-98611,TÍTULO: prototipo reciclaje inclusivo y colabo...,otro,otro
1,18IS6-98607,TÍTULO: sistema de reciclaje de frutas para co...,otro,otro
2,18IS6-98575,TÍTULO: escuela de cocina territorial y susten...,otro,otro
3,18IS6-98557,"TÍTULO: ""el partido de mi barrio rancagua"", la...",otro,otro
4,18IS6-98389,TÍTULO: modelo de mediación ambiental y certif...,otro,otro
...,...,...,...,...
8000,18IS9-97652,TÍTULO: rcicla2: programa de formación ambient...,otro,otro
8001,18IS6-98635,TÍTULO: senderismo y escalada para la superaci...,otro,otro
8002,18IS6-98632,TÍTULO: agroinm integremos a inmigrantes en el...,otro,otro
8003,18IS6-98630,TÍTULO: modelo de gestión participativa para l...,otro,otro


## Cruzar datos

In [5]:
df = df_corfo.merge(df_class, left_on="Código Proyecto", right_on="código").merge(df_health, left_on="Código Proyecto", right_on="código", suffixes=["_específica", "_salud"])
df

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,ERNC,Tendencia Final,código_específica,glosa_específica,clasificación_específica,clasificación_normalizada_específica,código_salud,glosa_salud,clasificación_salud,clasificación_normalizada_salud
0,23SN-242063,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77003755-7,Bebidas Saludables SpA,Mejorar las capacidades de las bebidas Masala ...,...,NaN,Sin tendencia,23SN-242063,TÍTULO: Mejorar las capacidades de las bebidas...,otro,otro,23SN-242063,TÍTULO: Mejorar las capacidades de las bebidas...,otro,otro
1,23SN-242058,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,76228136-8,Tecagromed SpA,TECFILMS: Desarrollo de un biomaterial en form...,...,NaN,Química Verde (Green Chemestry),23SN-242058,TÍTULO: TECFILMS: Desarrollo de un biomaterial...,otro,otro,23SN-242058,TÍTULO: TECFILMS: Desarrollo de un biomaterial...,otro,otro
2,23SN-242042,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77618332-6,Green Focus Solutions SpA,Kit molecular para la identificación rápida de...,...,NaN,Nanotecnología,23SN-242042,TÍTULO: Kit molecular para la identificación r...,otro,otro,23SN-242042,TÍTULO: Kit molecular para la identificación r...,otro,otro
3,23SN-242041,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77704466-4,Zeroglut SpA,Evaluación de tecnologías de envasado para inc...,...,NaN,Sin tendencia,23SN-242041,TÍTULO: Evaluación de tecnologías de envasado ...,otro,otro,23SN-242041,TÍTULO: Evaluación de tecnologías de envasado ...,otro,otro
4,23SN-242036,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77147033-5,La mar spa,Desarrollo de un pastel de jaibas listo para c...,...,NaN,Sin tendencia,23SN-242036,TÍTULO: Desarrollo de un pastel de jaibas list...,otro,otro,23SN-242036,TÍTULO: Desarrollo de un pastel de jaibas list...,otro,otro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8000,11IEI-12756,Desarrolla innovación,Subsidio,Innovación Empresarial Individual,Innovación Empresarial Individual,FINALIZADO,Persona Jurídica constituida en Chile,76059703-1,SOCIEDAD DE INVERSIONES SIMA LIMITADA,desarrollo de tintas de impresión inkjet de gr...,...,NaN,Sin tendencia,11IEI-12756,TÍTULO: desarrollo de tintas de impresión inkj...,otro,otro,11IEI-12756,TÍTULO: desarrollo de tintas de impresión inkj...,otro,otro
8001,11IEI-12162,Desarrolla innovación,Subsidio,Innovación Empresarial Individual,Innovación Empresarial Individual,FINALIZADO,Persona Jurídica constituida en Chile,96930530-5,VINOTEC CHILE S A,desarrollo de productos de alto valor agregado...,...,NaN,Sin tendencia,11IEI-12162,TÍTULO: desarrollo de productos de alto valor ...,otro,otro,11IEI-12162,TÍTULO: desarrollo de productos de alto valor ...,otro,otro
8002,11IEI-11916,Desarrolla innovación,Subsidio,Innovación Empresarial Individual,Innovación Empresarial Individual,FINALIZADO,Persona Jurídica constituida en Chile,96596370-7,AGUAMARINA SPA.,desarrollo de solución tecnológica scab (siste...,...,NaN,Sin tendencia,11IEI-11916,TÍTULO: desarrollo de solución tecnológica sca...,otro,otro,11IEI-11916,TÍTULO: desarrollo de solución tecnológica sca...,otro,otro
8003,11FBCT-12991,Desarrolla innovación con I+D,Subsidio,Financiamiento Basal Transitorio Para Consorci...,Consorcios Tecnológicos,FINALIZADO,Persona Jurídica constituida en Chile,76571080-4,INNOVACION Y TECNOLOGIA VITIVINICOLA S.A,continuidad operativa del consorcio tecnológic...,...,NaN,Sin tendencia,11FBCT-12991,TÍTULO: continuidad operativa del consorcio te...,otro,otro,11FBCT-12991,TÍTULO: continuidad operativa del consorcio te...,otro,otro


In [6]:
df_valid = df[df["Año Adjudicación"] >= 2013]
df_valid

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,ERNC,Tendencia Final,código_específica,glosa_específica,clasificación_específica,clasificación_normalizada_específica,código_salud,glosa_salud,clasificación_salud,clasificación_normalizada_salud
0,23SN-242063,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77003755-7,Bebidas Saludables SpA,Mejorar las capacidades de las bebidas Masala ...,...,NaN,Sin tendencia,23SN-242063,TÍTULO: Mejorar las capacidades de las bebidas...,otro,otro,23SN-242063,TÍTULO: Mejorar las capacidades de las bebidas...,otro,otro
1,23SN-242058,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,76228136-8,Tecagromed SpA,TECFILMS: Desarrollo de un biomaterial en form...,...,NaN,Química Verde (Green Chemestry),23SN-242058,TÍTULO: TECFILMS: Desarrollo de un biomaterial...,otro,otro,23SN-242058,TÍTULO: TECFILMS: Desarrollo de un biomaterial...,otro,otro
2,23SN-242042,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77618332-6,Green Focus Solutions SpA,Kit molecular para la identificación rápida de...,...,NaN,Nanotecnología,23SN-242042,TÍTULO: Kit molecular para la identificación r...,otro,otro,23SN-242042,TÍTULO: Kit molecular para la identificación r...,otro,otro
3,23SN-242041,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77704466-4,Zeroglut SpA,Evaluación de tecnologías de envasado para inc...,...,NaN,Sin tendencia,23SN-242041,TÍTULO: Evaluación de tecnologías de envasado ...,otro,otro,23SN-242041,TÍTULO: Evaluación de tecnologías de envasado ...,otro,otro
4,23SN-242036,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77147033-5,La mar spa,Desarrollo de un pastel de jaibas listo para c...,...,NaN,Sin tendencia,23SN-242036,TÍTULO: Desarrollo de un pastel de jaibas list...,otro,otro,23SN-242036,TÍTULO: Desarrollo de un pastel de jaibas list...,otro,otro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7225,12NIN-18079,Entorno para la innovación,Subsidio,Nodos Para La Innovación (I),Nodos,FINALIZADO,Persona Jurídica constituida en Chile,70008310-1,CAMARA NACIONAL DE COMERCIO SERVICIOS Y TURISM...,"red de apoyo a las pymes emergentes, acceso m...",...,NaN,Sin tendencia,12NIN-18079,"TÍTULO: red de apoyo a las pymes emergentes, ...",otro,otro,12NIN-18079,"TÍTULO: red de apoyo a las pymes emergentes, ...",otro,otro
7226,12NIN-18074,Entorno para la innovación,Subsidio,Nodos Para La Innovación (I),Nodos,FINALIZADO,Persona Jurídica constituida en Chile,65060484-9,ASOCIACIÓN DE MUNICIPALIDADES PARQUE CORDILLERA,nodo parques cordillera,...,NaN,Sin tendencia,12NIN-18074,TÍTULO: nodo parques cordillera. OBJETIVO: obj...,otro,otro,12NIN-18074,TÍTULO: nodo parques cordillera. OBJETIVO: obj...,otro,otro
7227,12NIN-18072,Entorno para la innovación,Subsidio,Nodos Para La Innovación (I),Nodos,FINALIZADO,Persona Jurídica constituida en Chile,65034978-4,"ASOCIACIÓN GREMIAL DE PROPIETARIOS, TENEDORES ...",nodo de emprendimientos sustentables en áreas ...,...,NaN,Sin tendencia,12NIN-18072,TÍTULO: nodo de emprendimientos sustentables e...,otro,otro,12NIN-18072,TÍTULO: nodo de emprendimientos sustentables e...,otro,otro
7228,12ETN-18828,Desarrolla innovación,Subsidio,Empaquetamiento Tecnológico Para Nuevos Negoci...,Empaquetamiento Tecnológico Para Nuevos Negocios,FINALIZADO,Persona Jurídica constituida en Chile,76128685-4,MAQUI NEW LIFE S.A.,inmunoestimulante de origen botánico para salm...,...,NaN,Sin tendencia,12ETN-18828,TÍTULO: inmunoestimulante de origen botánico p...,otro,otro,12ETN-18828,TÍTULO: inmunoestimulante de origen botánico p...,salud,salud


## Validación

### Casos raros

In [7]:
df_valid["clasificación_normalizada_específica"].value_counts()

otro                7117
cáncer                56
diabetes              29
cardiaca              27
diabetes, cáncer       1
Name: clasificación_normalizada_específica, dtype: int64

Hay un caso que tiene "diabetes, cáncer"; lo revisaremos a continuación.

In [8]:
df[df["clasificación_normalizada_específica"] == "diabetes, cáncer"]["Objetivo"].values

array(['contribuir a la ciencia médica moderna, pacientes e instituciones con una herramienta digital de entrenamiento científico internacional de medicina del estilo de vida, experimentada y adecuada a la realidad local por greenmann institute, institución pionera de iquique, respaldada a nivel global por instituciones médicas calificadas. lifestyle medicine es una disciplina nueva en chile y que actualmente hoy es una de las más poderosas intervenciones de atención médicas en el mundo, basada en evidencias científicas, para mejorar el acceso a la atención médica de los pacientes y de las instituciones de salud pública y privada de la región de tarapacá y chile, mediante una plataforma digital de entrenamiento en medicina del estilo de vida greenmann system para prevenir, tratar y revertir enfermedades no transmisibles o crónicas, como obesidad, diabetes cáncer, ecv, etc., basado en ciencia y tecnología aplicada a la vida saludable para conseguir un estilo de vida ideal sin enfermedad

Efectivamente, el proyecto trata de cáncer y también de diabetes. Para subsanar esto, duplicaremos ese proyecto, asignándole en una fila la clasificación "cáncer" y en otra "diabetes".

In [9]:
df_valid[df_valid["clasificación_normalizada_específica"] == "diabetes, cáncer"]

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,ERNC,Tendencia Final,código_específica,glosa_específica,clasificación_específica,clasificación_normalizada_específica,código_salud,glosa_salud,clasificación_salud,clasificación_normalizada_salud
3425,18IS1-97634,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76713787-7,CAPACITACION MAXIMILIANO GREGORY FUNES BASUALTO,plataforma digital de sistema greenmann medici...,...,NaN,Sin tendencia,18IS1-97634,TÍTULO: plataforma digital de sistema greenman...,"diabetes, cáncer","diabetes, cáncer",18IS1-97634,TÍTULO: plataforma digital de sistema greenman...,salud,salud


In [10]:
df_valid = df_valid.append(df_valid[df_valid["clasificación_normalizada_específica"] == "diabetes, cáncer"]).reset_index()
df_valid[df_valid["clasificación_normalizada_específica"] == "diabetes, cáncer"]

<ipython-input-10-a69f02f0cdb2>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_valid = df_valid.append(df_valid[df_valid["clasificación_normalizada_específica"] == "diabetes, cáncer"]).reset_index()


,index,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,...,ERNC,Tendencia Final,código_específica,glosa_específica,clasificación_específica,clasificación_normalizada_específica,código_salud,glosa_salud,clasificación_salud,clasificación_normalizada_salud
3425,3425,18IS1-97634,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76713787-7,CAPACITACION MAXIMILIANO GREGORY FUNES BASUALTO,...,NaN,Sin tendencia,18IS1-97634,TÍTULO: plataforma digital de sistema greenman...,"diabetes, cáncer","diabetes, cáncer",18IS1-97634,TÍTULO: plataforma digital de sistema greenman...,salud,salud
7230,3425,18IS1-97634,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76713787-7,CAPACITACION MAXIMILIANO GREGORY FUNES BASUALTO,...,NaN,Sin tendencia,18IS1-97634,TÍTULO: plataforma digital de sistema greenman...,"diabetes, cáncer","diabetes, cáncer",18IS1-97634,TÍTULO: plataforma digital de sistema greenman...,salud,salud


In [11]:
df_valid.loc[3425, "clasificación_normalizada_específica"] = "diabetes"
df_valid.loc[7230, "clasificación_normalizada_específica"] = "cáncer"
df_valid["clasificación_normalizada_específica"].value_counts()

otro        7117
cáncer        57
diabetes      30
cardiaca      27
Name: clasificación_normalizada_específica, dtype: int64

### Discrepancias entre ambas clasificaciones

Revisaremos aquellos casos donde para la clasificación salud vs otro quedó con la etiqueta `otro`, pero en la clasificación de enfermedades específicas sí quedó con alguna enfermedad (`cáncer`, `diabetes` o `cardiaca`)

In [13]:
with pd.option_context('display.max_colwidth', 3000):
  display(df_valid.loc[(df_valid["clasificación_normalizada_salud"] == "otro") &
                       (df_valid["clasificación_normalizada_específica"] != "otro"),
                       ["glosa_específica", "clasificación_normalizada_salud", "clasificación_normalizada_específica"]])

,glosa_específica,clasificación_normalizada_salud,clasificación_normalizada_específica
4872,"TÍTULO: agregación de valor a los descartes de cerezas de exportación mediante la generación de un alimento procesado. OBJETIVO: marcela correa es kinesióloga de profesión, sin embargo, siempre ha estado vinculada a la producción y explotación de cultivos de cerezas de exportación, actividad heredada de su padre, quién ha realizado actividades agrícolas por más de 40 años. en la actualidad, marcela posee 20 hectáreas de cultivo de cerezas de exportación, con una productividad promedio de 3 ton/hectárea. en la última temporada logró producir 62 toneladas, de las cuales, el 10% se perdió como descarte. la diferenciación que posee la cereza de exportación frente a las convencionales, es su gran tamaño y características organolépticas. esta diferenciación con respecto a la cereza o gunida que se comercializa a nivel nacional (de pequeño tamaño) se debe a las condiciones de manejo y cultivo antes de la cosecha, además de los fertilizantes utilizados en su producción. a su vez, al ser un producto de exportación, los cultivos deben ser tratados cuidadosamente, ya que pueden ser gravemente afectados por las heladas, lluvias, sequías u otros efectos negativos provocados por el clima, que inciden negativamente en la floración, y en la calidad organoléptica del producto. todo este cuidado excesivo que se le debe dar a la cereza de exportación provoca que los costos asociados al manejo y cuidado de los cultivos antes y después de la cosecha sean altísimos frente a otro tipo de cultivos frutales, y por tanto, cualquier problema en pre y pos cosecha de la cereza genera un producto de segunda calidad que en el mercado local no se logra pagar; y por tanto, todo el descarte producido son pérdidas directas para esta productora.no obstante lo anterior, y debido a que la cereza como tal es un alimento rico en flavonoides y antioxidantes, biomoléculas que pueden reducir el riesgo de padecer cáncer y enfermedades cardio-vasculares; además contiene vitamina a y c; y es rica en un aminoácido esencial en la dieta, el triptófano (constituyente esencial de las proteínas y precursor de la serotonina y de la melatonina),",otro,cáncer


Es sólo un caso.

### Validación por muestreo aleatorio

Seleccionaremos algunos casos al azar, los etiquetaremos manualmente y luego compararemos los resultados con la clasificación entregada por GPT 4.

Para realizar esto, se iteró varias veces hasta encontrar un conjunto de 30 muestras que satisficieran la condición de tener al menos una muestra clasificada como cada una de las enfermedades específicas. Al encontrar un conjunto que cumplía dicha condición, se fijó ese conjunto (los índices del dataframe) y se procedió a etiquetar manualmente las muestras.

In [16]:
indexes = [3555, 1432, 3324, 6833, 2411, 6791, 4881, 6511, 5023, 2965, 4019, 5770, 3625, 3182, 1981, 3250, 4074, 5071, 6665, 1136, 5079, 4059, 6464, 1050, 1377, 1244, 1044, 3742,   65, 6206]

In [21]:
df_sample = df_valid.iloc[indexes][["glosa_específica", "clasificación_normalizada_salud", "clasificación_normalizada_específica"]].copy().reset_index()
with pd.option_context('display.max_colwidth', 500):
  display(df_sample[["glosa_específica"]])

,glosa_específica
0,"TÍTULO: sistema de detección múltiple de los principales virus patógenos en cerdos. OBJETIVO: desarrollar herramientas de diagnóstico molecular de las principales patologías virales que afectan la producción de cerdos en chile, equilibrando rapidez, especificidad y costos."
1,"TÍTULO: 3D Printing para 6G y 5G. OBJETIVO: fabricar y validar técnica y comercialmente impresora 3d con sistema de aglutinación de resina con carga metálica de desarrollo propio, para producir piezas electrónicas y circuitos con estándares 5g y 6g (antenas, guías de onda) utilizados en la industria aeroespacial y telecomunicaciones terrestre. así, ocular 3d tendrá la capacidad de volverse un productor mundial de servicios de manufactura de estas piezas, aumentando su competitividad frente a..."
2,"TÍTULO: sistema remi. OBJETIVO: desarrollar un prototipo a escala real de un sistema de gestión para la operación en linea de plantas productivas de mediana minería (remi) que permita obtener en tiempo real toda la información relevante del proceso, para tomar las acciones y desiciones en forma oportuna, informada y con datos objetivos y reales del proceso. herramienta que debe ser flexible,escalable, con tecnología iot para lograr bajos costos de infraestructura it y accesible desde cualqui..."
3,"TÍTULO: proyecto de telemedida, respuesta a la autoridad y mejora de calidad de producto. OBJETIVO: adquirir e implementar un sistema de telemedida para medir, controlar y registrar consumo de energia."
4,"TÍTULO: meetbee: monitoreo en campo de polinizadores con inteligencia artificial y herramientas de gestión. OBJETIVO: optimizar maquinaria especializada para el monitoreo de abejas, esto mediante la incorporación de funcionalidades y diseño modular, que permitan el monitoreo en arboles, plantaciones, ciudades y casas, aumentando el rango de obtención de datos y la capacidad de entrenamiento la inteligencia artificial meetbee. estos desarrollos con la intención de incorporar valor a los clie..."
5,TÍTULO: contratación de ingeniero civil industrial para implementación de línea de proceso bovino en planta faenadora cisne austral ltda.. OBJETIVO: desarrollar los estudios técnicos para la implementación de la línea de faena bovina en la planta faenadora cisne austral ltda.
6,"TÍTULO: plataforma online para formación docente en aprendizaje social y emocional, personalizable y de impacto medible. OBJETIVO: crearemos una nueva modalidad de entrega online de nuestro servicio presencial de formación docente y coaching educativo en aprendizaje socio-emocional. con el fin de aumentar nuestro alcance y ventas, testear si los buenos resultados que obtenemos de manera presencial se obtienen en modalidad online y beneficiar a un mayor número de comunidades educativas en su ..."
7,TÍTULO: nuevo canal de venta directa de paquetes turísticos opcionales en hoteles. OBJETIVO: diseñar una aplicación de venta vía hoteles para acercar los servicios de protours a los pasajeros que requieren productos a medida
8,TÍTULO: formulación de nuevos jugos para la generación de alimentos saludables y funcionales. OBJETIVO: desarrollar una estrategia de formulación de jugos para la obtención de jugos saludables y funcionales
9,"TÍTULO: El programa Corvina, compuesto por el Programa de Diversificación de la Acuicultura Chilena, PDACH (desde el año 2010 hasta el año 2020), y el Programa Tecnológico Complementario, PTEC (desde el año 2016 hasta el año 2022)"


Anotamos las respuestas que nosotros daríamos para la clasificación salud vs otro, y también para la clasificación de enfermedades específicas.

In [25]:
health_answers = ["otro", "otro", "otro", "otro", "otro",
                  "otro", "otro", "otro", "salud", "otro",
                  "salud", "otro", "otro", "otro", "salud",
                  "salud", "otro", "otro", "otro", "otro",
                  "otro", "otro", "otro", "otro", "otro",
                  "otro", "otro", "salud", "otro", "otro"]
diseases_answers = ["otro", "otro", "otro", "otro", "otro",
                    "otro", "otro", "otro", "otro", "otro",
                    "otro", "otro", "otro", "otro", "diabetes",
                    "cardiaca", "otro", "otro", "otro", "otro",
                    "otro", "otro", "otro", "otro", "otro",
                    "otro", "otro", "cáncer", "otro", "otro"]

df_sample["respuestas_salud"] = health_answers
df_sample["respuestas_específicas"] = diseases_answers

df_sample

,index,glosa_específica,clasificación_normalizada_salud,clasificación_normalizada_específica,respuestas_salud,respuestas_específica,respuestas_específicas
0,3555,TÍTULO: sistema de detección múltiple de los p...,salud,otro,otro,otro,otro
1,1432,TÍTULO: 3D Printing para 6G y 5G. OBJETIVO: fa...,otro,otro,otro,otro,otro
2,3324,TÍTULO: sistema remi. OBJETIVO: desarrollar un...,otro,otro,otro,otro,otro
3,6833,"TÍTULO: proyecto de telemedida, respuesta a la...",otro,otro,otro,otro,otro
4,2411,TÍTULO: meetbee: monitoreo en campo de poliniz...,otro,otro,otro,otro,otro
5,6791,TÍTULO: contratación de ingeniero civil indust...,otro,otro,otro,otro,otro
6,4881,TÍTULO: plataforma online para formación docen...,otro,otro,otro,otro,otro
7,6511,TÍTULO: nuevo canal de venta directa de paquet...,otro,otro,otro,otro,otro
8,5023,TÍTULO: formulación de nuevos jugos para la ge...,salud,otro,salud,otro,otro
9,2965,"TÍTULO: El programa Corvina, compuesto por el ...",otro,otro,otro,otro,otro


#### Salud vs otro

In [23]:
with pd.option_context('display.max_colwidth', 500):
  display(df_sample.loc[df_sample["clasificación_normalizada_salud"] != df_sample["respuestas_salud"], ["glosa_específica", "clasificación_normalizada_salud", "respuestas_salud"]])

,glosa_específica,clasificación_normalizada_salud,respuestas_salud
0,"TÍTULO: sistema de detección múltiple de los principales virus patógenos en cerdos. OBJETIVO: desarrollar herramientas de diagnóstico molecular de las principales patologías virales que afectan la producción de cerdos en chile, equilibrando rapidez, especificidad y costos.",salud,otro
26,"TÍTULO: Upcycling en la industria hortofrutícola: snacks extruidos saludables en base a verduras descartadas. OBJETIVO: desarrollar prototipos de snacks extruidos en base a excedentes de la industria hortofrutícola chilena; con propiedades más saludables que los disponibles en el mercado actual y sin sellos de advertencia; desarrollándose a nivel piloto mediante tecnología de extrusión, junto a su envase sostenible; validados en términos productivos, de inocuidad, nutricionales y sensoriales...",salud,otro


Son sólo dos casos con discrepancias. De todas formas:

1. "Sistema de detección múltiple de los principales virus patógenos en cerdos". Este proyecto trata temas relacionados a salud, pero no humana.
2. "Upcycling en la industria hortofrutícola: snacks extruidos saludables en base a verduras descartadas." Este proyecto trata sobre crear snacks saludables, por lo cual se puede entender por qué GPT-4 piensa que sí es un proyecto relacionado a salud.

In [24]:
print(classification_report(df_sample["respuestas_salud"], df_sample["clasificación_normalizada_salud"]))

              precision    recall  f1-score   support

        otro       1.00      0.92      0.96        25
       salud       0.71      1.00      0.83         5

    accuracy                           0.93        30
   macro avg       0.86      0.96      0.90        30
weighted avg       0.95      0.93      0.94        30



#### Enfermedades específicas

In [26]:
with pd.option_context('display.max_colwidth', 500):
  display(df_sample.loc[df_sample["clasificación_normalizada_específica"] != df_sample["respuestas_específicas"], ["glosa_específica", "clasificación_normalizada_específica", "respuestas_específicas"]])

,glosa_específica,clasificación_normalizada_específica,respuestas_específicas


No hay ninguna discrepancia :)

In [27]:
print(classification_report(df_sample["respuestas_específicas"], df_sample["clasificación_normalizada_específica"]))

              precision    recall  f1-score   support

    cardiaca       1.00      1.00      1.00         1
      cáncer       1.00      1.00      1.00         1
    diabetes       1.00      1.00      1.00         1
        otro       1.00      1.00      1.00        27

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

